### Initialize notebook

In [8]:
# Torch imports
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset
import PIL.Image as IMG

# Other imports
import matplotlib.pyplot as plt
import numpy as np
import os

# NOTE: Aashis, you should use the training images to develop your algorithm.
# The test images should be reserved for validating its performance afterwards
# os.chdir('/home/ak/Spring2018/ature')
os.chdir('/home/akhanal1/Spring2018/ature')

sep = os.sep

# Define folders (create them if needed)
Dirs = {}
Dirs['train_data']      = 'data'+sep+'DRIVE'+sep+'training'+sep +'patches'
Dirs['test_data']      = 'data'+sep+'DRIVE'+sep+'test'+sep +'patches'

# Set up execution flags
Flags = {}
Flags['useGPU'] = False

In [9]:
class DriveDataset(Dataset):
    
    ## INPUT
    # data_path should contain pickled numpy array of dimension N * (D+1)
    # Where extra one dimension stores the correct lable among (0, 1, 2, 3)
    
    ## self.target contains one-hot encoding
    ## self.labels contains true labels (single value)
    
    def __init__(self, data_path=None, height=None, width=None, transform=None, load_one=False):
        
        self.data = None
        self.height = height
        self.width = width
        self.transform = transform
        
        for data_file in os.listdir(data_path):
            
            data_file = os.path.join(data_path, data_file)
            print('Data file: ' + data_file)
            if self.data is None:
                self.data = np.load(data_file)
            else:
                self.data = np.concatenate((self.data, np.load(data_file)), axis=0)
            
            if load_one:
                break
    
        self.data_len = self.data.shape[0]
        
        self.target = np.zeros((self.data_len, 4))
        self.labels = self.data[:, self.height * self.width] 
        self.target[range(self.data_len), self.labels] = 1
        
        self.data = self.data[:,0:self.height * self.width]
        self.labels = torch.from_numpy(self.labels)
        
    def __getitem__(self, index):
        
        a_data = self.data[index]
        img_arr = a_data.reshape(self.height, self.width)
        img = IMG.fromarray(img_arr)
        
        if self.transform is not None:
            img_tensor = self.transform(img)
            
        return (img_tensor, self.labels[index])
    
    def __len__(self):
        return self.data_len

In [10]:
transform = transforms.Compose([transforms.ToTensor()])

trainset = DriveDataset(data_path=Dirs['train_data'], height=31, width=31, transform=transform, load_one=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=2,
                                          shuffle=True, num_workers=1)
testset = DriveDataset(data_path=Dirs['test_data'], height=31, width=31, transform=transform, load_one=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=2,
                                          shuffle=True, num_workers=1)

classes = ('white', 'green', 'black', 'red')

Data file: data/DRIVE/training/patches/26_training.npy
Data file: data/DRIVE/test/patches/19_test.npy


In [11]:
IMG.fromarray(trainset.data[10000].reshape(31,31))

In [12]:
trainset.data.shape

(186895, 961)

### Define the network

In [13]:
class Net(nn.Module):
    def __init__(self, width):
        super(Net, self).__init__()
        nChannels = 1
        convKernelSize = 3
        convStride = 1
        mpKernelSize = 2
        mpStride = 2        
        padding = 0
        dilation = 1

        self.pool = nn.MaxPool2d(mpStride)

        self.conv1 = nn.Conv2d(nChannels, 6, convKernelSize)
        width = self.output_size(width,convKernelSize,padding,dilation,
            mpKernelSize,mpStride)
        self.conv2 = nn.Conv2d(6, 16, convKernelSize)
        width = self.output_size(width,convKernelSize,padding,dilation,
            mpKernelSize,mpStride)

        self.linearWidth = 16*int(width)*int(width)
        self.fc1 = nn.Linear(self.linearWidth, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 4)
        self.sm = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, self.linearWidth)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    # Output size of a convolution, followed by max pooling operation
    def output_size(self,inputSize,convKernelSize,padding,dilation,
                    kernelSize,stride):
        return np.floor(((inputSize - 2*np.floor(convKernelSize/2)) + 
                        2*padding - dilation*(kernelSize-1) - 1) / stride + 1)

width = 31
net = Net(width)

# Send network to the GPU, if requested
if Flags['useGPU']:
    net.cuda()

# Define loss criterion
# criterion = nn.L1Loss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


### Train the network

In [14]:
for epoch in range(2): 
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        if Flags['useGPU']:
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())                
        else:                
            inputs, labels = Variable(inputs), Variable(labels)
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


[1,  2000] loss: 0.716
[1,  4000] loss: 0.689
[1,  6000] loss: 0.705
[1,  8000] loss: 0.676
[1, 10000] loss: 0.658
[1, 12000] loss: 0.683
[1, 14000] loss: 0.661
[1, 16000] loss: 0.702
[1, 18000] loss: 0.660
[1, 20000] loss: 0.672
[1, 22000] loss: 0.600
[1, 24000] loss: 0.502
[1, 26000] loss: 0.536
[1, 28000] loss: 0.481
[1, 30000] loss: 0.457
[1, 32000] loss: 0.431
[1, 34000] loss: 0.422
[1, 36000] loss: 0.404
[1, 38000] loss: 0.392
[1, 40000] loss: 0.393
[1, 42000] loss: 0.388
[1, 44000] loss: 0.374
[1, 46000] loss: 0.363
[1, 48000] loss: 0.370
[1, 50000] loss: 0.365
[1, 52000] loss: 0.371
[1, 54000] loss: 0.362
[1, 56000] loss: 0.357
[1, 58000] loss: 0.330
[1, 60000] loss: 0.325
[1, 62000] loss: 0.327
[1, 64000] loss: 0.327
[1, 66000] loss: 0.312
[1, 68000] loss: 0.338
[1, 70000] loss: 0.313
[1, 72000] loss: 0.302
[1, 74000] loss: 0.324
[1, 76000] loss: 0.314
[1, 78000] loss: 0.314
[1, 80000] loss: 0.293
[1, 82000] loss: 0.299
[1, 84000] loss: 0.322
[1, 86000] loss: 0.300
[1, 88000] 

### Test the network

In [15]:
correct = 0
total = 0
for i, data in enumerate(testloader, 0):
    
    images, labels = data  
    if Flags['useGPU']:
        images = images.cuda()
        labels = labels.cuda()

    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
    if i % 2000 == 1999:
        print('Accuracy of %d test images: %d %%' % (i+1, 100 * correct / total))


Accuracy of 2000 test images: 88 %
Accuracy of 4000 test images: 89 %
Accuracy of 6000 test images: 89 %
Accuracy of 8000 test images: 89 %
Accuracy of 10000 test images: 88 %
Accuracy of 12000 test images: 88 %
Accuracy of 14000 test images: 89 %
Accuracy of 16000 test images: 88 %
Accuracy of 18000 test images: 88 %
Accuracy of 20000 test images: 88 %
Accuracy of 22000 test images: 88 %
Accuracy of 24000 test images: 88 %
Accuracy of 26000 test images: 88 %
Accuracy of 28000 test images: 88 %
Accuracy of 30000 test images: 88 %
Accuracy of 32000 test images: 88 %
Accuracy of 34000 test images: 88 %
Accuracy of 36000 test images: 88 %
Accuracy of 38000 test images: 88 %
Accuracy of 40000 test images: 88 %
Accuracy of 42000 test images: 88 %
Accuracy of 44000 test images: 88 %
Accuracy of 46000 test images: 88 %
Accuracy of 48000 test images: 88 %
Accuracy of 50000 test images: 88 %
Accuracy of 52000 test images: 88 %
Accuracy of 54000 test images: 88 %
Accuracy of 56000 test images: 8

### Check per-class performance

In [ ]:
class_correct = list(0. for i in range(4))
class_total = list(0. for i in range(4))
i = 0
for data in testloader:
    
    images, labels = data
    
    if Flags['useGPU']:
        images = images.cuda()
        labels = labels.cuda()

    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(2):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1
    i = i+1
    if i > 10000:
        break

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
